In [1]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
print(os.getcwd())

/content


In [2]:
class CricketMatchParser:
    def __init__(self, match_type, folder_path):
        self.match_type = match_type
        self.folder_path = folder_path

    def parse_match(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            match_data = json.load(f)
        file_name = os.path.basename(file_path)
        match_id = os.path.splitext(file_name)[0]

        info = match_data.get('info', {})
        match_metadata = {
            'match_id': match_id,
            'match_type': info.get('match_type', self.match_type),
            'venue': info.get('venue', ''),
            'city': info.get('city', ''),
            'date': info.get('dates', [''])[0],
            'team1': info.get('teams', ['',''])[0],
            'team2': info.get('teams', ['',''])[1],
            'toss_winner': info.get('toss', {}).get('winner', ''),
            'toss_decision': info.get('toss', {}).get('decision', ''),
            'winner': info.get('outcome', {}).get('winner', ''),
            'overs': info.get('overs', ''),
            'season': info.get('season', '')
        }

        delivery_data = []
        innings = match_data.get('innings', [])
        for inning in innings:
            batting_team = inning.get('team', '')
            overs = inning.get('overs', [])
            for over_data in overs:
                over_number = over_data.get('over')
                deliveries = over_data.get('deliveries', [])
                for ball_number, delivery in enumerate(deliveries, start=1):
                    delivery_entry = {
                        **match_metadata,
                        'batting_team': batting_team,
                        'over': over_number,
                        'ball': ball_number,
                        'batter': delivery.get('batter', ''),
                        'bowler': delivery.get('bowler', ''),
                        'non_striker': delivery.get('non_striker', ''),
                        'runs_batter': delivery.get('runs', {}).get('batter', 0),
                        'runs_extras': delivery.get('runs', {}).get('extras', 0),
                        'runs_total': delivery.get('runs', {}).get('total', 0),
                        'extras_type': ','.join(delivery.get('extras', {}).keys()) if 'extras' in delivery else '',
                        'wicket_type': '',
                        'player_out': ''
                    }

                    if 'wickets' in delivery:
                        for wicket in delivery['wickets']:
                            delivery_entry['wicket_type'] = wicket.get('kind', '')
                            delivery_entry['player_out'] = wicket.get('player_out', '')

                    delivery_data.append(delivery_entry)

        df = pd.DataFrame(delivery_data)
        return df,match_id

In [3]:
class CricketDataTransformer:
    def __init__(self, match_type, folder_path):
        self.match_type = match_type
        self.folder_path = folder_path
        self.all_deliveries = []

    def load_all_json_files(self):
        parser = CricketMatchParser(self.match_type, self.folder_path)
        for file in os.listdir(self.folder_path):
            if file.endswith(".json"):
                file_path = os.path.join(self.folder_path, file)
                try:
                    deliveries_df, match_id = parser.parse_match(file_path)
                    if not deliveries_df.empty:
                        self.all_deliveries.append(deliveries_df)
                except Exception as e:
                    print(f"Failed to parse {file}: {e}")

    def get_combined_dataframe(self):
        if self.all_deliveries:
            return pd.concat(self.all_deliveries, ignore_index=True)
        return pd.DataFrame()

    def save_to_csv(self, output_file):
        df = self.get_combined_dataframe()
        if not df.empty:
            df.to_csv(output_file, index=False)
            print(f"Saved CSV: {output_file}")
        else:
            print(f"No data to save for {output_file}")

In [4]:
import zipfile
match_types = ['Test', 'ODI', 'T20', 'IPL']
zip_paths = [
    r'/content/tests_json.zip',
    r'/content/odis_json.zip',
    r'/content/t20s_json.zip',
    r'/content/ipl_json.zip'
]
extract_folders = [
    r'/content/tests_json',
    r'/content/odis_json',
    r'/content/t20s_json',
    r'/content/ipl_json'
]

for match_type, zip_path, extract_to in zip(match_types, zip_paths, extract_folders):
    if not os.path.exists(extract_to):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Extracted {match_type} JSON files to: {extract_to}")
    else:
        print(f"{match_type} folder already exists, skipping extraction.")

    transformer = CricketDataTransformer(match_type,extract_to)
    transformer.load_all_json_files()
    transformer.save_to_csv(f'{match_type}.csv')

Extracted Test JSON files to: /content/tests_json
Saved CSV: Test.csv
Extracted ODI JSON files to: /content/odis_json
Saved CSV: ODI.csv
Extracted T20 JSON files to: /content/t20s_json
Saved CSV: T20.csv
Extracted IPL JSON files to: /content/ipl_json
Saved CSV: IPL.csv


In [5]:
df = pd.read_csv("/content/IPL.csv")

In [6]:
df.head()

,match_id,match_type,venue,city,date,team1,team2,toss_winner,toss_decision,winner,...,ball,batter,bowler,non_striker,runs_batter,runs_extras,runs_total,extras_type,wicket_type,player_out
0,1304090,T20,"Dr DY Patil Sports Academy, Mumbai",Navi Mumbai,2022-04-30,Rajasthan Royals,Mumbai Indians,Mumbai Indians,field,Mumbai Indians,...,1,JC Buttler,DR Sams,D Padikkal,1,0,1,NaN,NaN,NaN
1,1304090,T20,"Dr DY Patil Sports Academy, Mumbai",Navi Mumbai,2022-04-30,Rajasthan Royals,Mumbai Indians,Mumbai Indians,field,Mumbai Indians,...,2,D Padikkal,DR Sams,JC Buttler,1,0,1,NaN,NaN,NaN
2,1304090,T20,"Dr DY Patil Sports Academy, Mumbai",Navi Mumbai,2022-04-30,Rajasthan Royals,Mumbai Indians,Mumbai Indians,field,Mumbai Indians,...,3,JC Buttler,DR Sams,D Padikkal,1,0,1,NaN,NaN,NaN
3,1304090,T20,"Dr DY Patil Sports Academy, Mumbai",Navi Mumbai,2022-04-30,Rajasthan Royals,Mumbai Indians,Mumbai Indians,field,Mumbai Indians,...,4,D Padikkal,DR Sams,JC Buttler,0,0,0,NaN,NaN,NaN
4,1304090,T20,"Dr DY Patil Sports Academy, Mumbai",Navi Mumbai,2022-04-30,Rajasthan Royals,Mumbai Indians,Mumbai Indians,field,Mumbai Indians,...,5,D Padikkal,DR Sams,JC Buttler,0,0,0,NaN,NaN,NaN


In [7]:
! pip install mysql-connector-python sqlalchemy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 52.0 MB/s eta 0:00:00


In [8]:
! pip install numpy

In [9]:
from sqlalchemy import create_engine, text
import mysql.connector
import pandas as pd

class Database:
    def __init__(self, host, user, password, port, database):
        self.host = host
        self.user = user
        self.password = password
        self.port = port
        self.database = database
        self.connection = None
        self.cursor = None

    def connect(self):
        self.connection = mysql.connector.connect(
            host=self.host,
            user=self.user,
            password=self.password,
            port=self.port,
            connection_timeout=300,
            read_timeout=300,
            write_timeout=300,
            autocommit=True
        )
        self.cursor = self.connection.cursor()
        print("Connection established.")
        self.cursor.execute("SET GLOBAL max_allowed_packet=536870912")  # 512MB
        self.cursor.execute("SET SESSION wait_timeout=28800")
        self.create_database()
        self.cursor.execute(f"USE {self.database}")
        self.connection.commit()

    def create_database(self):
        sql = f"CREATE DATABASE IF NOT EXISTS {self.database}"
        self.cursor.execute(sql)
        self.connection.commit()

    def create_tables(self):
        create_table_queries = [
            """
            CREATE TABLE IF NOT EXISTS test_matches (
                match_id INT AUTO_INCREMENT PRIMARY KEY,
                match_type VARCHAR(20),
                venue VARCHAR(255),
                city VARCHAR(100),
                date DATE,
                team1 VARCHAR(100),
                team2 VARCHAR(100),
                toss_winner VARCHAR(100),
                toss_decision VARCHAR(10),
                winner VARCHAR(100),
                overs INT,
                season VARCHAR(20),
                batting_team VARCHAR(100),
                `over` INT,
                ball INT,
                batter VARCHAR(100),
                bowler VARCHAR(100),
                non_striker VARCHAR(100),
                runs_batter INT,
                runs_extras INT,
                runs_total INT,
                extras_type VARCHAR(100),
                wicket_type VARCHAR(100),
                player_out VARCHAR(100)
            );
            """,
            """
            CREATE TABLE IF NOT EXISTS odi_matches (
                delivery_id INT AUTO_INCREMENT PRIMARY KEY,
                match_id VARCHAR(100),
                match_type VARCHAR(20),
                venue VARCHAR(255),
                city VARCHAR(100),
                date DATE,
                team1 VARCHAR(100),
                team2 VARCHAR(100),
                toss_winner VARCHAR(100),
                toss_decision VARCHAR(10),
                winner VARCHAR(100),
                overs INT,
                season VARCHAR(20),
                batting_team VARCHAR(100),
                `over` INT,
                ball INT,
                batter VARCHAR(100),
                bowler VARCHAR(100),
                non_striker VARCHAR(100),
                runs_batter INT,
                runs_extras INT,
                runs_total INT,
                extras_type VARCHAR(100),
                wicket_type VARCHAR(100),
                player_out VARCHAR(100)
            );
            """,
            """
            CREATE TABLE IF NOT EXISTS t20_matches (
                delivery_id INT AUTO_INCREMENT PRIMARY KEY,
                match_id VARCHAR(100),
                match_type VARCHAR(20),
                venue VARCHAR(255),
                city VARCHAR(100),
                date DATE,
                team1 VARCHAR(100),
                team2 VARCHAR(100),
                toss_winner VARCHAR(100),
                toss_decision VARCHAR(10),
                winner VARCHAR(100),
                overs INT,
                season VARCHAR(20),
                batting_team VARCHAR(100),
                `over` INT,
                ball INT,
                batter VARCHAR(100),
                bowler VARCHAR(100),
                non_striker VARCHAR(100),
                runs_batter INT,
                runs_extras INT,
                runs_total INT,
                extras_type VARCHAR(100),
                wicket_type VARCHAR(100),
                player_out VARCHAR(100)
            );
            """,
            """
            CREATE TABLE IF NOT EXISTS ipl_matches (
                delivery_id INT AUTO_INCREMENT PRIMARY KEY,
                match_id VARCHAR(100),
                match_type VARCHAR(20),
                venue VARCHAR(255),
                city VARCHAR(100),
                date DATE,
                team1 VARCHAR(100),
                team2 VARCHAR(100),
                toss_winner VARCHAR(100),
                toss_decision VARCHAR(10),
                winner VARCHAR(100),
                overs INT,
                season VARCHAR(20),
                batting_team VARCHAR(100),
                `over` INT,
                ball INT,
                batter VARCHAR(100),
                bowler VARCHAR(100),
                non_striker VARCHAR(100),
                runs_batter INT,
                runs_extras INT,
                runs_total INT,
                extras_type VARCHAR(100),
                wicket_type VARCHAR(50),
                player_out VARCHAR(100)
            );
            """
        ]

        for ddl in create_table_queries:
            try:
                self.cursor.execute(ddl)
                print("Table created or already exists.")
            except Exception as e:
                print(f"Error creating table: {e}")
        self.connection.commit()

db = Database(host="gateway01.us-west-2.prod.aws.tidbcloud.com",user="45zomgjzrEaJ3az.root", password="pL7yBHrDIMx652G0", port=4000, database="cricsheet")
db.connect()
db.create_tables()

Connection established.
Table created or already exists.
Table created or already exists.
Table created or already exists.
Table created or already exists.


In [10]:
import pandas as pd
import mysql.connector
import numpy as np

def insert_match_data(cursor, connection, df, table_name, batch_size=100):
    expected_columns = [
        'match_id', 'match_type', 'venue', 'city', 'date', 'team1', 'team2',
        'toss_winner', 'toss_decision', 'winner', 'overs', 'season',
        'batting_team', 'over', 'ball', 'batter', 'bowler', 'non_striker',
        'runs_batter', 'runs_extras', 'runs_total', 'extras_type', 'wicket_type', 'player_out'
    ]

    missing = [col for col in expected_columns if col not in df.columns]
    if missing:
        print(f"Missing columns in `{table_name}` DataFrame: {missing}")
        return

    df = df.replace({np.nan: None, 'nan': None})
    df = df.dropna(subset=['match_id'])
    df = df.drop_duplicates(subset=['match_id'])
    fill_values = {
        'match_type': 'unknown',
        'venue': 'unknown',
        'city': 'unknown',
        'team1': 'unknown',
        'team2': 'unknown',
        'toss_winner': 'unknown',
        'toss_decision': 'unknown',
        'winner': 'unknown',
        'season': 'unknown',
        'batting_team': 'unknown',
        'batter': 'unknown',
        'bowler': 'unknown',
        'non_striker': 'unknown',
        'extras_type': 'none',
        'wicket_type': 'none',
        'player_out': 'none',
        'runs_batter': 0,
        'runs_extras': 0,
        'runs_total': 0,
        'overs': 0,
        'over': 0,
        'ball': 0
    }

    df.fillna(value=fill_values, inplace=True)

    insert_query = f"""
        INSERT IGNORE INTO {table_name} (
            match_id, match_type, venue, city, date, team1, team2,
            toss_winner, toss_decision, winner, overs, season,
            batting_team, `over`, ball, batter, bowler, non_striker,
            runs_batter, runs_extras, runs_total, extras_type, wicket_type, player_out
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    print(f"Starting data insertion into table `{table_name}`...")

    try:
        all_data = [
            (
                row.get('match_id'), row.get('match_type'), row.get('venue'), row.get('city'), row.get('date'),
                row.get('team1'), row.get('team2'), row.get('toss_winner'), row.get('toss_decision'), row.get('winner'),
                row.get('overs'), row.get('season'), row.get('batting_team'), row.get('over'), row.get('ball'),
                row.get('batter'), row.get('bowler'), row.get('non_striker'), row.get('runs_batter'), row.get('runs_extras'),
                row.get('runs_total'), row.get('extras_type'), row.get('wicket_type'), row.get('player_out')
            ) for _, row in df.iterrows()
        ]

        total_rows = len(all_data)
        for start in range(0, total_rows, batch_size):
            end = start + batch_size
            batch = all_data[start:end]
            cursor.executemany(insert_query, batch)
            connection.commit()
            print(f"Inserted rows {start+1} to {min(end, total_rows)}")

        print(f"Data insertion complete for table `{table_name}`")

    except Exception as e:
        print(f"Error inserting into `{table_name}`: {e}")
        connection.rollback()

In [12]:
if __name__ == "__main__":
    try:
        df_ipl = pd.read_csv("IPL.csv", low_memory=False)
        df_test = pd.read_csv("Test.csv", low_memory=False)
        df_odi = pd.read_csv("ODI.csv", low_memory=False)
        df_t20 = pd.read_csv("T20.csv", low_memory=False)

        connection = mysql.connector.connect(
            host="gateway01.us-west-2.prod.aws.tidbcloud.com",
            user="45zomgjzrEaJ3az.root",
            password="pL7yBHrDIMx652G0",
            port="4000",
            database="cricsheet")

        cursor = connection.cursor()

        insert_match_data(cursor, connection, df_ipl, table_name="ipl_matches", batch_size=100)
        insert_match_data(cursor, connection, df_test, table_name="test_matches", batch_size=100)
        insert_match_data(cursor, connection, df_odi, table_name="odi_matches", batch_size=100)
        insert_match_data(cursor, connection, df_t20, table_name="t20_matches", batch_size=100)

        cursor.close()
        connection.close()

    except Exception as e:
        print(f"Fatal error: {e}")


Starting data insertion into table `ipl_matches`...
Inserted rows 1 to 100
Inserted rows 101 to 200
Inserted rows 201 to 300
Inserted rows 301 to 400
Inserted rows 401 to 500
Inserted rows 501 to 600
Inserted rows 601 to 700
Inserted rows 701 to 800
Inserted rows 801 to 900
Inserted rows 901 to 1000
Inserted rows 1001 to 1100
Inserted rows 1101 to 1169
Data insertion complete for table `ipl_matches`
Starting data insertion into table `test_matches`...
Inserted rows 1 to 100
Inserted rows 101 to 200
Inserted rows 201 to 300
Inserted rows 301 to 400
Inserted rows 401 to 500
Inserted rows 501 to 600
Inserted rows 601 to 700
Inserted rows 701 to 800
Inserted rows 801 to 869
Data insertion complete for table `test_matches`
Starting data insertion into table `odi_matches`...
Inserted rows 1 to 100
Inserted rows 101 to 200
Inserted rows 201 to 300
Inserted rows 301 to 400
Inserted rows 401 to 500
Inserted rows 501 to 600
Inserted rows 601 to 700
Inserted rows 701 to 800
Inserted rows 801 to 9